In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import time
from sklearn import metrics
from scipy.optimize import brute, minimize
from sklearn.metrics import make_scorer
from joblib import dump, load
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# if str(device) == 'cpu':
#     from sklearn.svm import SVR
#     from sklearn.linear_model import RidgeCV, Ridge, Lasso
# else:
#     from cuml.svm import SVR
#     from cuml.linear_model import Ridge
#     import cuml

In [2]:
def get_score(outputs, targets):
    mcrmse = []
    for i in range(2):
        mcrmse.append(
            metrics.mean_squared_error(
                targets[:, i],
                outputs[:, i],
                squared=False,
            ),
        )
    mcrmse = np.mean(mcrmse)
    return mcrmse


def obj_func(x, *oof_list):
    
    
    
    oof = 0
    x[x<0] = 0
    x = x/np.sum(x)
    for i in range(min(len(x), len(oof_list))):
        oof += x[i]*oof_list[i][['pred_0', 'pred_1']]
    ground_truth = oof_list[0][['content', 'wording']]
    return get_score(oof.values, ground_truth.values)

In [3]:
pred_cols = ['pred_0', 'pred_1']
target_cols = ['content', 'wording']

In [4]:
Data_Path = "C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/CommonLit - Evaluate Student Summaries/input/commonlit-evaluate-student-summaries/"
oof_024 = pd.read_csv('../024/Comonlist_024oof_df.csv', usecols=['student_id', 'fold']+pred_cols)

oof_024 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024, on='student_id')
print(oof_024.shape)

oof_024_2 = pd.read_csv('../024_2/Comonlist_024_2_oof_df.csv', usecols=['student_id', 'fold']+pred_cols)
oof_024_2 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024_2, on='student_id')
print(oof_024_2.shape)

oof_024_3 = pd.read_csv('../024_3/Comonlist_024_3_oof_df.csv', usecols=['student_id', 'fold']+pred_cols)
oof_024_3 = pd.merge(pd.read_csv(f'{Data_Path}summaries_train.csv'), oof_024_3, on='student_id')
print(oof_024_3.shape)


oof_tuple = tuple([oof_024, oof_024_2, oof_024_3])

for i in range(len(oof_tuple)):
    print(get_score(oof_tuple[i][['pred_0', 'pred_1']].values, oof_tuple[0][['content', 'wording']].values))
    

st = time.time()
res = minimize(obj_func, x0=np.ones((len(oof_tuple), )), args=oof_tuple, method='Nelder-Mead')
print((time.time()-st), 'sec.')

print('The optimal weights are', res.x)
print('The optimal OOF MCRMSE is', res.fun)


(7165, 8)
(7165, 8)
(7165, 8)
0.5597027507453362
0.5605429571763885
0.5578805947858928
0.20545053482055664 sec.
The optimal weights are [0.81914641 1.32156335 0.85428778]
The optimal OOF MCRMSE is 0.5520873341760086


In [5]:
w = res.x
sum_w = sum(w)
norm_w = [w/sum_w for w in w]
print(sum(norm_w))
w = [w/sum_w for w in w]
w

1.0


[0.27350486989087774, 0.4412569057496202, 0.2852382243595021]